In [ ]:
import os
import pandas as pd
import subprocess
from subprocess import call
import rasterio
import xarray as xr
import glob
from datetime import datetime, timedelta
import warnings

In [ ]:
def make_dataframe(mypath):
    hdf_name = []
    for image in os.listdir(mypath):
        if image.endswith('hdf'):
            hdf_name.append(image)
    dates = []
    for name in hdf_name:
        dates.append(name.split('.')[1])
    df_images = pd.DataFrame(list(zip(hdf_name, dates)), columns = ['image', 'date']) 
    grouped = df_images.groupby('date')['image'].apply(list).reset_index()
    grouped['image'].count()
    return grouped

In [ ]:
def mosaic_and_clip_lst(date,paths,timeofday):
    os.chdir('/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/lst_8day/composites_2020_2021')
    st_name = date+'_'+timeofday+'_merge.tif'
    print(st_name)
    merge_command = ['gdal_merge.py', "-o",st_name, ' '.join(paths)]
    merge_command = ' '.join(merge_command)
    #cmd_a = 'gdal_merge.py -o '+ st_name'-ul_lr 19.46 34.44 28.29 41.9 '
    subprocess.call(merge_command,shell=True)
    warp_command = ['gdalwarp','-t_srs','EPSG:4326','-te 19.46 34.44 28.29 41.9', st_name,st_name.split('.')[0]+'_warp.tif']
    warp_command = ' '.join(warp_command)
    subprocess.call(warp_command,shell=True)
    call(["rm", st_name])

In [ ]:
def create_lst_day(year,grouped):
    for index,row in grouped.iterrows():#['image'].loc[0]:
        paths = []
        for d in row['image']: 
            path = 'HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/lst_8day/'+str(year)+'/'+d+'":MODIS_Grid_8Day_1km_LST:LST_Day_1km'
            paths.append(path)
            #mosaic_and_clip_lst(path[0],paths,timeofday)
        date = d.split('.')[1]
        timeofday = 'day'
        mosaic_and_clip_lst(date,paths,timeofday)

In [ ]:
def create_lst_night(year,grouped):
    for index,row in grouped.iterrows():#['image'].loc[0]:
        paths = []
        for d in row['image']: 
            path = 'HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/lst_8day/'+str(year)+'/'+d+'":MODIS_Grid_8Day_1km_LST:LST_Night_1km'
            paths.append(path)
            #mosaic_and_clip_lst(path[0],paths,timeofday)
        date = d.split('.')[1]
        timeofday = 'night'
        mosaic_and_clip_lst(date,paths,timeofday)

In [ ]:
years =[2020,2021]
#[2010,2011,2012,2013,2014,2015,2016,2017,2018]

In [ ]:
for year in years:
    mypath = "/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/lst_8day/"+str(year)
    os.chdir(mypath)
    grouped = make_dataframe(mypath)
    create_lst_day(year,grouped)
    create_lst_night(year,grouped)

In [ ]:
grouped.loc[2]['image']

# NDVI

In [ ]:
/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi

In [ ]:
def make_dataframe(mypath):
    hdf_name = []
    for image in os.listdir(mypath):
        if image.endswith('hdf'):
            hdf_name.append(image)
    dates = []
    for name in hdf_name:
        dates.append(name.split('.')[1])
    df_images = pd.DataFrame(list(zip(hdf_name, dates)), columns = ['image', 'date']) 
    grouped = df_images.groupby('date')['image'].apply(list).reset_index()
    grouped['image'].count()
    return grouped

In [ ]:
def mosaic_and_clip_ndvi(date,paths,mode=''):
    os.chdir('/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_composites')
    st_name = date+'_'+mode+'_merge.tif'
    print(st_name)
    merge_command = ['gdal_merge.py', "-o",st_name, ' '.join(paths)]
    merge_command = ' '.join(merge_command)
    print(merge_command)
    #cmd_a = 'gdal_merge.py -o '+ st_name'-ul_lr 19.46 34.44 28.29 41.9 '
    subprocess.call(merge_command,shell=True)
    warp_command = ['gdalwarp','-t_srs','EPSG:4326','-te 19.46 34.44 28.29 41.9', st_name,st_name.split('.')[0]+'_warp.tif']
    warp_command = ' '.join(warp_command)
    subprocess.call(warp_command,shell=True)
    call(["rm", st_name])

In [ ]:
def create_ndvi_evi(year,grouped):
    for index,row in grouped.iterrows():#['image'].loc[0]:
        paths_ndvi = []
        paths_evi = []
        for d in row['image']: 
            path_ndvi = 'HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/'+ d +'":MODIS_Grid_16DAY_500m_VI:"500m 16 days NDVI"'
            path_evi = 'HDF4_EOS:EOS_GRID:"/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi/'+ d +'":MODIS_Grid_16DAY_500m_VI:"500m 16 days EVI"'
            paths_ndvi.append(path_ndvi)
            paths_evi.append(path_evi)
            #mosaic_and_clip_lst(path[0],paths,timeofday)
        date = d.split('.')[1]
        timeofday = 'day'
        mosaic_and_clip_ndvi(date,paths_ndvi,'ndvi')
        mosaic_and_clip_ndvi(date,paths_evi,'evi')

In [ ]:
for year in years:
    mypath = "/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_ndvi"
    os.chdir(mypath)
    grouped = make_dataframe(mypath)
    create_ndvi_evi(year,grouped)
    #create_lst_night(year,grouped

# TIF to nc

In [ ]:
veg_path = '/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/modis_composites'
lst_path = '/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/lst_8day/composites_2020_2021'

In [ ]:
veg_dest_path = '/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/veg'

In [ ]:
lst_dest_path = '/media/sg/91d62d44-8446-4f66-8327-bc09e774cbb1/home/df/Projects/datacube_project/lst'

In [ ]:
evi_files = glob.glob(os.path.join(veg_path, 'A*_evi_merge_warp.tif'))
ndvi_files = glob.glob(os.path.join(veg_path, 'A*_ndvi_merge_warp.tif'))
lst_day_files = glob.glob(os.path.join(lst_path, 'A*_day_merge_warp.tif'))
lst_night_files = glob.glob(os.path.join(lst_path, 'A*_night_merge_warp.tif'))

# ndvi

In [ ]:
data_dict = {}
for evi_file in evi_files:
    date_str = os.path.basename(evi_file)[1:8]
    if date_str in data_dict:
        data_dict[date_str].append((evi_file, None))
    else:
        data_dict[date_str] = [(evi_file, None)]
        
for ndvi_file in ndvi_files:
    date_str = os.path.basename(ndvi_file)[1:8]
    if date_str in data_dict:
        for i, (evi_file, ndvi_file_) in enumerate(data_dict[date_str]):
            if ndvi_file_ is None:
                data_dict[date_str][i] = (evi_file, ndvi_file)
                break

In [ ]:
for date_str, file_pairs in data_dict.items():
    doy = date_str[4:]
    year = int(date_str[:4])
    date = datetime(year, 1, 1) + timedelta(int(doy) - 1)
    yyyymmdd = date.strftime('%Y%m%d')
    nc_file_name = os.path.join(veg_dest_path, f'{yyyymmdd}_veg.nc')
    print(nc_file_name)
    datasets = []
    for evi_file, ndvi_file in file_pairs:
        if ndvi_file is None:
            print(f"No matching NDVI file found for EVI file {evi_file}")
            continue
        evi_da = xr.open_rasterio(evi_file).squeeze()
        ndvi_da = xr.open_rasterio(ndvi_file).squeeze()
        evi_da.name = 'evi'
        ndvi_da.name = 'ndvi'
        ds = xr.merge([evi_da, ndvi_da])
        datasets.append(ds)
    if datasets:
        combined_ds = xr.concat(datasets, dim='band')
        combined_ds = combined_ds.squeeze()
        combined_ds = combined_ds.drop('band')
        combined_ds = combined_ds.assign_coords(time=yyyymmdd)
        combined_ds = combined_ds.expand_dims('time')
        for attr_name in combined_ds.attrs:
            del ds.attrs[attr_name]
        combined_ds.to_netcdf(nc_file_name)

# lst

In [ ]:
data_lst_dict = {}
for day_file in lst_day_files:
    date_str = os.path.basename(day_file)[1:8]
    if date_str in data_lst_dict:
        data_lst_dict[date_str].append((day_file, None))
    else:
        data_lst_dict[date_str] = [(day_file, None)]

for night_file in lst_night_files:
    date_str = os.path.basename(night_file)[1:8]
    if date_str in data_lst_dict:
        for i, (day_file, night_file_) in enumerate(data_lst_dict[date_str]):
            if night_file_ is None:
                data_lst_dict[date_str][i] = (day_file, night_file)
                break

In [ ]:
warnings.filterwarnings("ignore", message="open_rasterio is Deprecated in favor of rioxarray")

In [ ]:
for date_str, file_pairs in data_lst_dict.items():
    datasets = []
    doy = date_str[4:]
    year = int(date_str[:4])
    date = datetime(year, 1, 1) + timedelta(int(doy) - 1)
    yyyymmdd = date.strftime('%Y%m%d')
    nc_file_name = os.path.join(lst_dest_path, f'{yyyymmdd}_lst.nc')
    print(nc_file_name)
    for lst_day, lst_night in file_pairs:
        if lst_night is None:
            print(f"No matching LST file found for night file {lst_night}")
            continue
        day_da = xr.open_rasterio(lst_day).squeeze()
        night_da = xr.open_rasterio(lst_night).squeeze()
        day_da.name = 'day'
        night_da.name = 'night'
        ds = xr.merge([day_da, night_da])
        datasets.append(ds)
    if datasets:
        combined_ds = xr.concat(datasets, dim='band')
        combined_ds = combined_ds.squeeze()
        combined_ds = combined_ds.drop('band')
        combined_ds = combined_ds.assign_coords(time=yyyymmdd)
        combined_ds = combined_ds.expand_dims('time')
        for attr_name in combined_ds.attrs:
            del ds.attrs[attr_name]
        combined_ds.to_netcdf(nc_file_name)